In [1]:
import time
import paho.mqtt.client
import paho.mqtt.publish
import numpy as np
from numpy import random
import datetime
import ssl
import sys
import psycopg2
from sqlalchemy import create_engine
import json

In [2]:
host='lallah.db.elephantsql.com'
user ='yirsxctz'
password='AS3sYxp2fpRwAczZa1OZpIzI4etWHM6n'
dbname='yirsxctz'

In [3]:
def myQuery(query):
    cur = myConnection.cursor()
    try:
        cur.execute(query)
    except Exception as e:
        myConnection.commit()
        print('Error en el query:', e)
    else:
        cur.close()
        myConnection.commit()
        return 1

In [4]:
myConnection = psycopg2.connect(host = host,
                                user= user, password =password,
                                dbname= dbname)

In [5]:
def on_connect(client, userdata, flags, rc):    
    print('connected (%s)' % client._client_id)
    client.subscribe(topic='Tienda/#', qos = 0)       

def on_message(client, userdata, message):   
    a = json.loads(message.payload)
    #warnings(a)
    insert_db(a)
    print(a)
    print('------------------------------')  

In [8]:
def insert_db(info):
    query = """INSERT INTO visitas_conocidas (id_cliente_afiliado, id_tienda, puntos, fecha)
                    values ("""+str(info["cliente"])+""",
                            """+str(info["tienda"])+""",
                            """+str(1)+""",
                            '"""+str(info["fecha"])+"""')"""

    if(int(info["afiliado"]) == 1):
        myQuery(query)
    
    for x in range(0,len(info["productos"])):
        query = """insert into compra (id_factura, id_producto, cantidad_producto,
                                    id_cambio_precio_producto, id_tienda, id_productoxestante)
                values ('"""+str(3)+"""',
                        '"""+str((info["productos"])[x])+"""',
                        '"""+str((info["cantidades"])[x])+"""',
                        '"""+str((info["cambios_precio_productos"])[x])+"""',
                        '"""+str(info["tienda"])+"""',
                        '"""+str((info["productosxestante"])[x])+"""')"""
        
        myQuery(query)

In [9]:
host = "broker.hivemq.com"

client = paho.mqtt.client.Client()

client.on_connect = on_connect

client.message_callback_add('Tienda/EntradaClientes', on_message)

client.connect(host=host) 

client.loop_start()

connected (b'')
Error en el query:Error en el query: column "id_cuenta_bancaria" does not exist
LINE 1: ...tal + (precio_producto)*NEW.cantidad_total  WHERE id_cuenta_...
                                                             ^
QUERY:  UPDATE factura SET subtotal = subtotal + (precio_producto)*NEW.cantidad_total  WHERE id_cuenta_bancaria = NEW.id_cuenta_bancaria
CONTEXT:  PL/pgSQL function actualizar_subtotal() line 9 at SQL statement

{'afiliado': 0, 'cliente': 1, 'tienda': 2, 'fecha': '2020-11-10 20:48:51.740253', 'id_factura': 1, 'productos': [1], 'cantidades': [3], 'cambios_precio_productos': [2], 'productosxestante': [1]}
------------------------------
 current transaction is aborted, commands ignored until end of transaction block

{'afiliado': 0, 'cliente': 1, 'tienda': 2, 'fecha': '2020-11-10 20:48:51.740253', 'id_factura': 1, 'productos': [1], 'cantidades': [3], 'cambios_precio_productos': [2], 'productosxestante': [1]}
------------------------------
